In [2]:
import tensorflow as tf
import numpy as np

# Time Windows
First, we will train a model to forecast the next step given the previous 20 steps, therefore, we need to create a dataset of 20-step windows for training.

In [4]:
dataset = tf.data.Dataset.range(10)
for i in dataset:
    print(i.numpy())

0
1
2
3
4
5
6
7
8
9


In [13]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1)
for i in dataset:
    for j in i:
        print(j.numpy(), end=" ")
    print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 
6 7 8 9 
7 8 9 
8 9 
9 


In [14]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
for i in dataset:
    for j in i:
        print(j.numpy(), end=" ")
    print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 


In [20]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
for i in dataset:
    print(i.numpy())

[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]
[5 6 7 8 9]


In [32]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=16)
for i, y in dataset:
    print(i.numpy(), y.numpy())

[3 4 5 6] [7]
[2 3 4 5] [6]
[0 1 2 3] [4]
[1 2 3 4] [5]
[5 6 7 8] [9]
[4 5 6 7] [8]


In [47]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=16)
for i,y in dataset:
    print(i.numpy(), y.numpy())

[5 6 7 8] [9]
[0 1 2 3] [4]
[3 4 5 6] [7]
[2 3 4 5] [6]
[4 5 6 7] [8]
[1 2 3 4] [5]


In [51]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=16)
dataset = dataset.batch(2).prefetch(1)
for i,y in dataset:
    print("X: ",i.numpy()) 
    print("Y: ",y.numpy())

X:  [[3 4 5 6]
 [1 2 3 4]]
Y:  [[7]
 [5]]
X:  [[2 3 4 5]
 [4 5 6 7]]
Y:  [[6]
 [8]]
X:  [[5 6 7 8]
 [0 1 2 3]]
Y:  [[9]
 [4]]


In [52]:
# Now Creating a function
def window_dataset(series, window_size, batch_size = 32, shuffle_buffer =16):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(buffer_size=16)
    dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset